In [1]:
import spacy
import re
from spacy.pipeline import EntityRuler
from spacy.matcher import Matcher
from spacy.tokens import Span
from spacy import displacy
from nav_pii_anon.spacy.spacy_model import SpacyModel
from nav_pii_anon.regex_engine import fnr
from nav_pii_anon.regex_engine.tlfnr import RegexTlfNr
from nav_pii_anon.regex_engine.date_time import RegexDateTime

## Using the Entity Ruler object from SpaCy

### Assigning multiple custom entities

In [2]:
TRAIN_DATA = [
    (
        "Horses are too tall and they pretend to care about your feelings",
        {"entities": [(0, 6, 'TLF')]},
    ),
    ("Do they bite?", {"entities": []}),
    (
        "horses are too tall and they pretend to care about your feelings",
        {"entities": [(0, 6, 'TLF')]},
    ),
    ("horses pretend to care about your feelings", {"entities": [(0, 6, 'TLF')]}),
    (
        "they pretend to care about your feelings, those horses",
        {"entities": [(48, 54, 'TLF')]},
    ),
    ("horses?", {"entities": [(0, 6, 'TLF')]}),
]

In [3]:
nlp = SpacyModel()
#nlp.train(TRAIN_DATA)
nlp.add_patterns()
nlp.disable_ner()

In [4]:
tx="Jeg er johanne Nymark og mitt fødselsnummer er 15044216652 og tlf er +47 99 11 92 34 . Jeg kommer innom 20062020 for å \
snakke om min 100 prosent stilling 99119234, og bergen, norge"
type(tx)

str

In [5]:
doc = nlp.get_doc(tx)
nlp.display_predictions(tx)

NameError: name 'options' is not defined

In [ ]:
nlp.replace(tx, shuffle=True)